In [23]:
import datetime
import re
import configparser
import os

import psycopg2

import fetch

In [111]:
import importlib 
importlib.reload(fetch)

<module 'fetch' from '/home/michel/scraper-place/fetch.py'>

In [24]:
#base_dir = os.path.dirname(os.path.realpath(__file__))
base_dir = os.getcwd()  # notebook
base_dir

'/home/michel/scraper-place'

In [25]:
# Read config

config = configparser.ConfigParser()
config_path = os.path.join(base_dir, 'config.ini')
config.read(config_path)
database_host = config['database']['host']
database_port = config['database']['port']
database_name = config['database']['name']
database_username = config['database']['username']
database_password = config['database']['password']
assert database_host == 'localhost'
assert database_port == '1234'

In [102]:
# Open connection
connection = psycopg2.connect(dbname=database_name, user=database_username, password=database_password)
cursor = connection.cursor()

In [103]:
def process_link(link, connection, cursor):
    annonce_id, org_acronym = re.match(fetch.link_regex, link).groups()
    
    cursor.execute("SELECT annonce_id, org_acronym FROM dce WHERE annonce_id = %s AND org_acronym = %s;", (annonce_id, org_acronym))
    results = cursor.fetchall()
    
    if results:
        return
    
    (annonce_id, org_acronym, links_boamp,
        reference, intitule, objet, reglement_ref,
        filename_reglement, reglement, filename_complement,
        complement, filename_avis, avis, filename_dce, dce) = fetch.fetch_data(link)
    now = datetime.datetime.now()
    
    cursor.execute("""
        INSERT INTO dce (
            annonce_id, org_acronym, links_boamp,
            reference, intitule, objet,
            reglement_ref, filename_reglement, filename_complement, filename_avis, filename_dce,
            fetch_datetime, reglement, complement, avis, dce
            )
            VALUES (
            %s, %s, %s,
            %s, %s, %s,
            %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s
           )""",
        (
            annonce_id, org_acronym, links_boamp,
            reference, intitule, objet,
            reglement_ref, filename_reglement, filename_complement, filename_avis, filename_dce,
            now, reglement, complement, avis, dce,
            )
        )
    connection.commit()


In [105]:
#links = fetch.fetch_current_annonces()

In [ ]:
for link in links:
    process_link(link, connection, cursor)

In [101]:
cursor.close()
connection.close()

In [78]:
# complement mais pas de reglement :
link

'https://www.marches-publics.gouv.fr/?page=entreprise.EntrepriseDetailConsultation&refConsultation=339793&orgAcronyme=g7h'

In [108]:
link

'https://www.marches-publics.gouv.fr/?page=entreprise.EntrepriseDetailConsultation&refConsultation=310484&orgAcronyme=t9y'